## Indicating how top 13F filers have performed over time

#### ALL in vs Cash Weighted (Sold out of position(s))

In [141]:
import pandas as pd
import requests
import yfinance as yf
import os
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [142]:
holdings = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Holdings_shares\Stanley_Druckenmiller.csv")

mapping1 = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Final_cusip_map\CUSIP_Mapping.csv", usecols = ["CUSIP", "SYMBOL"])
mapping2 = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Final_cusip_map\CUSIP_Mapping-Brocks-Laptop.csv", usecols = ["CUSIP", "SYMBOL"])
mapping = pd.concat([mapping1, mapping2]).drop_duplicates()

dates = [col for col in holdings.columns if '-' in col]

In [143]:
directory = "Holdings_shares"

# Create an empty dictionary to store DataFrames
dfs = {}

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df_name = os.path.splitext(filename)[0]  # Get the file name without the extension

        read_df = pd.read_csv(filepath)
        read_df = read_df.merge(mapping[["CUSIP", "SYMBOL"]].drop_duplicates(), how = 'left', on = "CUSIP")
        dfs[df_name] = read_df


## Uniue symbols used to pull sector
symbols = pd.concat(dfs.values())

symbols.loc[symbols['SYMBOL'] == 'BRKB', 'SYMBOL'] = 'BRK-B'

unique_symbols = symbols[["NAME OF ISSUER", "SYMBOL"]].drop_duplicates()
unique_symbols = unique_symbols[~unique_symbols["SYMBOL"].isnull()]

In [144]:
# Define start and end dates
start_date = "2013-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

# Create a list to store the downloaded dataframes
historical_data = []

for symbol in unique_symbols["SYMBOL"]:
    # Download historical data for the current symbol
    ticker_data = yf.download(symbol, start=start_date, end=end_date)

    # Append the data to the list
    historical_data.append(ticker_data)

# Optionally, concatenate the dataframes into a single DataFrame with a multi-index
historical_data = pd.concat(historical_data, keys=unique_symbols["SYMBOL"], names=["SYMBOL"]).reset_index()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['VRX']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2013-01-01 -> 2024-09-27)')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******************

In [145]:
test = holdings.merge(mapping, on = "CUSIP", how='left')
test = test[test.columns[1:]]

In [167]:
def quarterly_alpha(test):
    test1 = test.melt(id_vars=['NAME OF ISSUER', 'CUSIP', 'CALL', 'SYMBOL'], var_name='Date', value_name='Shares').drop_duplicates()
    test1["Date"] = pd.to_datetime(test1["Date"])

    ### adding 
    # Get unique dates from historical_data
    historical_dates = historical_data['Date'].drop_duplicates().sort_values()
    q_dates = test1[["Date", "CALL"]].drop_duplicates()

    # Define a function to find the closest date after a given date
    def find_closest_after(date, dates):
        return dates[dates > date].min()
    # Apply the function to each row in test1
    q_dates['NextDay'] = q_dates['Date'].apply(lambda x: find_closest_after(x, historical_dates))


    ## getting date of next market open
    test1 = test1.merge(q_dates[["Date", "NextDay"]], on = 'Date', how='left')

    test1.drop(columns="Date",inplace=True)


    test1 = test1.merge(historical_data[["SYMBOL","Date", "Open"]], left_on= ["SYMBOL", "NextDay"], right_on= ["SYMBOL", "Date"], how = 'left').drop_duplicates()
    test1["Starting Value"] = (test1["Open"] * test1["Shares"]).fillna(0)

    test1 = test1[(test1["Starting Value"] > 0) & (test1["CALL"].isna())]

    test1["Quarter"] = np.where(test1["Date"].dt.month <= 3, "Q1",
                            np.where(test1["Date"].dt.month <= 6, "Q2",
                                    np.where(test1["Date"].dt.month <= 9, "Q3","Q4")))

    test1["Year"] = test1["Date"].dt.year
    test1.drop(columns="Date",inplace=True)

    test1["Year_Quarter"] = test1["Year"].astype(str) + "-" + test1["Quarter"] # np.where(test1["Quarter"] == "Q4", (test1["Year"]-1).astype(str) + "-" + test1["Quarter"], test1["Year"].astype(str) + "-" + test1["Quarter"])

    ## joining Quarter end date 
    date_table = test1["NextDay"].drop_duplicates().reset_index()
    date_table = date_table.sort_values('NextDay', ascending = False)
    date_table["QuarterEnd"] = np.where(date_table['NextDay'].shift(1).isna(), historical_dates[len(historical_dates)-1].strftime('%Y-%m-%d'), date_table['NextDay'].shift(1).dt.strftime('%Y-%m-%d'))
    date_table["QuarterEnd"]  = pd.to_datetime(date_table["QuarterEnd"])


    test1 = test1.merge(date_table[["NextDay", "QuarterEnd"]], how = 'left', on = 'NextDay')

    ## Next Quarter opening price (Theoretical selling price)
    historical_data2 = historical_data.rename(columns={"Open": "Open_Next_Quarter"})
    test1 = test1.merge(historical_data2[["SYMBOL","Date", "Open_Next_Quarter"]], left_on= ["SYMBOL", "QuarterEnd"], right_on= ["SYMBOL", "Date"], how = 'left').drop_duplicates()
    test1["Ending Value"] = (test1["Open_Next_Quarter"] * test1["Shares"]).fillna(0)

    spy_index = yf.download('SPY', start=start_date, end=end_date).reset_index()

    returns = test1.groupby(["Year_Quarter", "NextDay", "QuarterEnd"])[["Starting Value", "Ending Value"]].sum().reset_index()
    spy = spy_index[["Date", "Open"]]
    spy.columns = ["Date", "spy_start"]
    spy["spy_end"] = spy["spy_start"]

    returns = returns.merge(spy[["Date", "spy_start"]], left_on = 'NextDay', right_on='Date', how = 'left')
    returns.drop(columns = 'Date', inplace = True)
    returns = returns.merge(spy[["Date", "spy_end"]], left_on = 'QuarterEnd', right_on='Date', how = 'left')
    returns.drop(columns = 'Date', inplace = True)

    returns["Quarterly Return"] = (returns["Ending Value"] - returns["Starting Value"])/returns["Starting Value"]
    returns["Benchmark Return"] = (returns["spy_end"] - returns["spy_start"])/returns["spy_start"]


    returns["Return Ann Roll4"] = (returns["Quarterly Return"] + 1).rolling(window=4).apply(lambda x: x.prod(), raw=True)
    returns["Benchmark Return Ann Roll4"] = (returns["Benchmark Return"] + 1).rolling(window=4).apply(lambda x: x.prod(), raw=True)

    returns["Total Return"] = (returns["Quarterly Return"] + 1).cumprod()
    returns["Benchmark Total Return"] = (returns["Benchmark Return"] + 1).cumprod()

    returns["Alpha"] = returns["Return Ann Roll4"] - returns["Benchmark Return Ann Roll4"]
    
    return returns[["Year_Quarter", "Alpha"]]

In [153]:

# Get unique dates from historical_data
historical_dates = historical_data['Date'].drop_duplicates().sort_values()
q_dates = test1[["NextDay", "Quarter"]].drop_duplicates()

# Define a function to find the closest date after a given date
def find_closest_after(date, dates):
    return dates[dates > date].min()

# Apply the function to each row in test1
q_dates['Closest_Historical_Date'] = q_dates['NextDay'].apply(lambda x: find_closest_after(x, historical_dates))

In [154]:
q_dates

,NextDay,Quarter,Closest_Historical_Date
0,2024-08-15,Q3,2024-08-16
109,2024-05-16,Q2,2024-05-17
228,2024-02-15,Q1,2024-02-16
324,2023-11-15,Q4,2023-11-16
413,2023-08-15,Q3,2023-08-16
511,2023-05-16,Q2,2023-05-17
604,2023-02-15,Q1,2023-02-16
692,2022-11-15,Q4,2022-11-16
796,2022-08-16,Q3,2022-08-17
860,2022-05-17,Q2,2022-05-18
